<a href="https://colab.research.google.com/github/Chaa00/Text-generation-LSTM/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text generation with LSTM** 



La premiere chose dont on a besoin c est d avoir un fichier text contenant des données enormes de mots afin d apprendre notre model de langage. Dans cet exemple nous allons utiliser des écrits de Nietzsche. 

In [ ]:
# loading libraries 
import keras
import numpy as np
#telecharger le fichier et le convertir en miniscules 
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


Dans la partie qui suive nous allons extraire des sequences de longueur qui se chevauchent partiellement "maxlen" ,  les encoder "one-hot" et les emballer dans un "3D Numpy array" x de shape (sequences, maxlen, unique_characters)  , on prepare par la suite an array y contenant ce 'one hot encoded' qui viennent juste apres chaque sequence extraite. 

In [ ]:
# Longueur des sequences de caracteres extraites
maxlen = 60
# Nous echantillonnons une nouvelle sequence de caractere a chaque etape
step = 3

# Ceci contient nos sequences extraites 
sentences = []
# ceci contient notre cible
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# liste des caracteres uniques dans le fichier 
chars = sorted(list(set(text)))
print(chars)

print('Unique characters:', len(chars))

# un dictionnaire qui relie les caractères uniques a leurs indexes dans 'chars'
char_indices = dict((char, chars.index(char)) for char in chars)

# one-hot encode les caracteres into binary arrays 
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
Unique characters: 57
Vectorization...


# **Building the network**

Notre réseau est une LSTM couche unique suivie d un Dense classificateur et d un softmax sur tous les caracteres possibles

In [ ]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# **Training the language model and sampling from it**


 ""     1 Draw from the model a probability distribution for the next character, given the generated text available so far.


2 Reweight the distribution to a certain temperature.


3 Sample the next character at random according to the reweighted distribution.

4 Add the new character at the end of the available text. ""

In [ ]:
# c'est le code que nous utilisons pour reponderer la distribution de la probabilite originale sortant du modele, 
#et en tirer un index de caractere  "sampling function":
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Par la suite nous utilisions une boucle où nous entraînons et generons le texte a plusieurs reprises. Nous commençons a generer du texte en utilisant une plage de temperatures differentes apres chaque epoque. Cela nous permet de voir comment le texte generé evolue lorsque le modele commence a converger, ainsi que l'impact de la temperature dans la strategie d'echantillonnage

In [ ]:
import random
import sys
# Ici nous pouvons voir le contenu de chaque epochs et comment ca fonctionne (on a 5 epochs for run )
for epoch in range(1, 5):
    print('epoch', epoch)
    # Fit the model pour 1 epoch on the "training data"  disponible
    model.fit(x, y,batch_size=128, epochs=1)

    # selectionner une partie du text au hasard 
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # on genere 400 caracteres 
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 270s 172ms/step - loss: 1.4351
--- Generating with seed: "ditional
estimates of things, as the most desirable of all s"
------ temperature: 0.2
ditional
estimates of things, as the most desirable of all seek on the sensible of the subtle and the subjection of the spirit and all the world and the sension of the spirit and the suppose of the present and some the such a subtle the striff the such as the spirit and the stronger of the subtle the strength the conscience, and the stronger of the superior of the same and the subtle that the superior the the subjection of the present the sension of the pr
------ temperature: 0.5
rior the the subjection of the present the sension of the problem for the ething and one and the master of such an the present, the conscience, it really and the person to the soul and what the most for the sension of the contempt the lack of the heart and sympathy of the conscience,
and the man and the profound and 

# **Conclusion**

*Une température basse entraîne un texte  répétitif et prévisible.



*Une température elevée entraine un texte plus intéressant, surprenant; il peut parfois inventer des mots complètement nouveaux. La structure locale commence à se décomposer et la plupart des mots ressemblent à des chaînes semi-aléatoires de caractères.

=> 0,5 est la température la plus intéressante pour la génération du texte 